In [ ]:
from cases.att48 import att48
from cases.bayg29 import bayg29
from cases.bays29 import bays29
from cases.berlin52 import berlin52
from cases.br17 import br17
from cases.brazil58 import brazil58
from cases.burma14 import burma14
from cases.dantzig42 import dantzig42
from cases.fri26 import fri26
from cases.ft53 import ft53
from cases.ft70 import ft70
from cases.ftv33 import ftv33
from cases.ftv35 import ftv35
from cases.ftv38 import ftv38
from cases.ftv44 import ftv44
from cases.ftv47 import ftv47
from cases.ftv55 import ftv55
from cases.ftv64 import ftv64
from cases.ftv70 import ftv70
from cases.gr17 import gr17
from cases.gr21 import gr21
from cases.gr48 import gr48
from cases.hk48 import hk48
from cases.p43 import p43
from cases.pr76 import pr76
from cases.ry48p import ry48p
from cases.st70 import st70
from cases.swiss42 import swiss42
from cases.ulysses16 import ulysses16
from cases.ulysses22 import ulysses22

cases = [
    burma14,  # 14
    ulysses16,  # 16
    br17,  # 17
    gr17,  # 17
    gr21,  # 21
    ulysses22,  # 22
    fri26,  # 26
    bayg29,  # 29
    bays29,  # 29
    ftv33,  # 34
    ftv35,  # 36
    ftv38,  # 39
    dantzig42,  # 42
    swiss42,  # 42
    p43,  # 43
    ftv44,  # 45
    att48,  # 48
    ftv47,  # 48
    gr48,  # 48
    hk48,  # 48
    ry48p,  # 48
    berlin52,  # 52
    ft53,  # 53
    ftv55,  # 56
    brazil58,  # 58
    ftv64,  # 65
    ft70,  # 70
    st70,  # 70
    ftv70,  # 71
    pr76,  # 76
]

In [ ]:
import os
import json
from dadk.BinPol import *

print(f"Python {sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}");
print(f"DADK {DADK_version} {DADK_flavour}");

loop_counter = 0
with_error = True
while with_error:
    with_error = False
    loop_counter += 1

    for case in cases:

        id, N, objective, penalty = case()

        src_filename = f'results-{id}.json'
        tgt_filename = f'results-{id}-short.json' 

        if not os.path.isfile(src_filename):
            if loop_counter ==1:
                print(f"can't find {src_filename}")
            continue
    
        if (not os.path.isfile(tgt_filename)) or (os.path.getmtime(src_filename) > os.path.getmtime(tgt_filename)):
    
            print(f'id {id} ... ', end='') 
            
            try:
                with open(src_filename) as fp:
                    data = json.load(fp)
                with open(src_filename.replace('.json', '-backup.json'), 'w') as fp:
                    json.dump(data, fp)
                for M in sorted(data.keys()):
                    for seed_id in sorted(data[M].keys()):
                        if '_data' in seed_id:
                            continue
                        data[M][seed_id] = [ (entry[0], int(M)*penalty.compute(entry[2])) for entry in data[M][seed_id] ]
    
                with open(tgt_filename, 'w') as fp:
                    json.dump(data, fp)
                
                print('done')    
                
            except Exception as oops:
                print('error', oops)
                with_error = True


print('\nready')

In [ ]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from tabulate import tabulate
import json
import os
import math
import numpy as np
from typing import List           

html = []

html.append('<html>')
html.append('<body>')
html.append('<table>')

for case in cases:

    id, N, objective, penalty = case()    
   
    filename = os.path.join('.', f'results-{id}-short.json' )
    if not os.path.isfile(filename):
        continue        
        
    with open(filename, 'r') as fp:
        summary = json.load(fp)  
        objective = summary.get('objective', [])
        summary = {int(M):{seed:summary[M][seed] for seed in summary[M]} for M in summary}

    seed_ids = []
    for M in summary:
        for seed_id in summary[M]:
            if '_data' not in seed_id:
                if seed_id not in seed_ids:
                    seed_ids.append(seed_id)
    seed_ids = sorted(list(set(seed_ids)))
 
    values = []
    for M in summary:
        for seed in seed_ids:
            if seed in summary[M]:
                values += [entry[0] for entry in summary[M][seed] if entry[1] == 0]
    distance = min(values) if len(values) > 0 else 1
    
    print('distance', distance)
       
    fig, ax = plt.subplots(1, 1, figsize=(12,4)  )
    ax2 = ax.twinx()
    fig.canvas.header_visible = False
   
    for M in summary:

        values = []
        counter = 0
        for seed in seed_ids:
            if seed in summary[M]:
                values += [entry[0] / distance for entry in summary[M][seed] if entry[1] == 0]
                counter += len(summary[M][seed])
                
        if counter > 0:
            ax.scatter( [M], [100*len(values)/counter], c='blue', s=10)
            if len(values)>0:
                ax2.scatter( [M], np.min(values), c='green', s=10)
                ax2.scatter( [M], np.mean(values), c='red', s=10)   
    
    epsilon = 0.05
    
    ax2.set_ylabel("rel. mean energy",color="red",fontsize=14)

    ylim2 = (1.0, 2.0)
    delta2 = epsilon*(ylim2[1]-ylim2[0])
    ax2.set_ylim((ylim2[0] - delta2, ylim2[1] + delta2))        
    ax.set_ylabel("% valid solutions",color="blue",fontsize=14)
    ylim1 = (0.0, 100.0)
    delta1 = epsilon*(ylim1[1]-ylim1[0])
    ax.set_ylim((ylim1[0] - delta1, ylim1[1] + delta1))
    ax.set_xlabel(f"factor for penalty [{min(summary.keys()):,d} - {max(summary.keys()):,d}]", color="black",fontsize=14)
    plt.title(f"{id} {N:,d} cities, {N*N:,d} bits, {(N*N)*(N*N):,d} iterations")
    png_filename = filename.replace('.json', '.png')
    plt.savefig(png_filename, dpi=150)
    plt.show()
    plt.close()

    html.append(f'\t<tr>')
    html.append(f'\t\t<td><h1>{id}</h1></td><td><a href="{png_filename}"><img src="{png_filename}" width="33%" height="33%"/></a></td>')
    html.append(f'\t</tr>')

html.append('</table>')
html.append('</body>')
html.append('</html>')

with open('summary.html', 'w') as fp:
    fp.write('\n'.join(html))